# Final Excercise

In this notebook, you will find the last exercise of the lecture:
You are handed an initial dataset with several features and a univariate target. Next, you have to decide how to proceed. Since you do not have enough data to yet construct a classifier/predictor of any sensible evaluation metrics, the first task is, therefore, to acquire more data. For this purpose you can obtain batches of data according to your own design of experiments, so you will need to decide which experiments you consider necessary to perform. 

You will have four opportunities to acquire more data. Each time you have to decide which experiments to run and send those to Franz Götz-Hahn as a CSV file. The deadlines are 16.06.2023, 23.06.2023, 30.06.2023, and 07.07.2023 and 12:00 (noon). The format in all cases is a table with one row for each choosable feature, and the column entries corresponding to the desired values. Each individual sample will take approximately 30min, so pick a reasonable amount of experiments. For example, you will get the result for 100 experiments roughly 50 hours after the respective deadline. Should the experiment not be conductible, you will get a ``None`` as a result, e.g., if a feature value is out of range.

Once you have your data, you should compare the performance of different classifiers in predicting the targets. The classifiers to compare are [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), and [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html). You should utilize all the different parts of the E2ML lecture that you considern appropriate. This could include Data Preprocessing, Design of Experiments for the batches, deciding on Performance Measures, Statistical Significance Testing of a hypothesis, Design of Experiments for Hyperparameter Optimization.

Should you wish to present the results from this excercise in the oral examination, you need to hand in your entire package until 14.07.2023-23:59 as a GitHub Repository. Send the link to the (public) repository to Franz Götz-Hahn via [E-Mail](mailto:franz.goetz-hahn@uni-kassel.de). Please use the README of the repository to describe the structure of the package, include any required packages in the setup.py, add the data in the data subfolder, save any results in the results subfolder, and include a _descriptive_ jupyter notebook in the notebooks subfolder.

Do note, that the point of this excercise is **not** to achieve the best performance of your models, but rather to document your process and give the motivation behind your chosen approaches, _even the ones that failed_.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from e2ml.experimentation import perform_bayesian_optimization
from e2ml.preprocessing import PrincipalComponentAnalysis
from sklearn.decomposition import PCA# used because at the time of writing this code the solutions weren't available and I wanted to make sure the code is right
from e2ml.experimentation import acquisition_ei
from e2ml.models import GaussianProcessRegression

### **Mollusc Classification** <a class="anchor" id="heart"></a>

Your dataset describes some physical measurements of a specific type of molluscs. Your goal is to predict the `Stage of Life` of the mollusc. The data you can get looks as follows:


| Sex	|Length	|Width	|Height|	Weight	|Non_Shell Weight	|Intestine Weight	|Shell Weight	|Stage of Life |
| ---                           | ----   | ----    | ----    | ----   |----             |----    |---- | ---------- |
| {Male (M), Female (F), Indeterminate (I)} | float (inches)     | float (inches)     |  float (inches)     | float (gram)      | float (gram)              | float (gram)     |  float (gram)     | {Child, Adolescent (Adole), Adult}      |

The table headings are identical to the column names in the corresponding CSV-files. 

We can send out divers that look for molluscs that fit your needs, which will subsequently be analyzed in a laboratory. You can request molluscs with all features except the Stage of Life attribute, as it is the target. The first day of diving has already been completed. After 8 hours of diving, they brought up the following molluscs:

In [2]:
initial_molluscs_data = pd.read_csv('../data/initial_molluscs_data.csv')
initial_molluscs_data

,Sex,Length,Width,Height,Weight,Non_Shell Weight,Intestine Weight,Shell Weight,Stage of Life
0,F,0.450,0.345,0.120,0.4165,0.1655,0.0950,0.1350,Adult
1,F,0.475,0.380,0.145,0.5700,0.1670,0.1180,0.1870,Adole
2,M,0.610,0.485,0.170,1.0225,0.4190,0.2405,0.3600,Adult
3,I,0.430,0.340,0.105,0.4405,0.2385,0.0745,0.1075,Adole
4,M,0.205,0.155,0.045,0.0425,0.0170,0.0055,0.0155,Adult
5,M,0.600,0.475,0.175,1.3445,0.5490,0.2875,0.3600,Child
6,I,0.515,0.390,0.110,0.5310,0.2415,0.0980,0.1615,Adult
7,F,0.625,0.495,0.160,1.1115,0.4495,0.2825,0.3450,Child
8,F,0.650,0.520,0.195,1.6275,0.6890,0.3905,0.4320,Adult
9,F,0.620,0.480,0.165,1.0430,0.4835,0.2210,0.3100,Adult


## **This Notebook only contains the first prototyping and creation of the first batch of data, it is not pretty but it is what I initally did so i kept it in. For the actual code I used to get the third to last batch look into new_data_pipeline.ipynb**

In [6]:
#group data by Stage of Life
adults = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Adult"]
adoles = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Adole"]
children = initial_molluscs_data.loc[initial_molluscs_data["Stage of Life"] == "Child"]
# just get the single columns
full_length = initial_molluscs_data["Length"]
full_width = initial_molluscs_data["Width"]
full_height = initial_molluscs_data["Height"]
full_weight = initial_molluscs_data["Weight"]
full_non_shell_weight = initial_molluscs_data["Non_Shell Weight"]
full_intestine_weight = initial_molluscs_data["Intestine Weight"]
full_shell_weight = initial_molluscs_data["Shell Weight"]
full_sexes = initial_molluscs_data["Sex"]
full_stages = initial_molluscs_data["Stage of Life"]

# compute statistics about the relation of different features
volume = full_length * full_width * full_height
weight_volume_quotients = full_weight / volume 
non_shell_quotient = full_non_shell_weight / full_weight
intestine_quotient = full_intestine_weight / full_weight
shell_quotient = full_shell_weight / full_weight

def getOneHotEncoding(data):
    values = np.sort(np.unique(data))
    enc = np.zeros((len(data), len(values)))
    for i, x in enumerate(data):
        enc[i, np.where(values == x)[0][0]] = 1
    return enc

def score_cross_entropy_loss(mdl, x, y):
    if(len(y.shape) == 1):
        y = getOneHotEncoding(y)
    y_pred = softmax(mdl.predict_proba(x))
    #return cross_entropy_loss(y, y_pred)
    return [log_loss(y[i], y_pred[i])*-1 for i in range(len(y_pred))]

def cross_entropy_loss(y_true, y_pred):
    #if y contains the information of which class is true as index
    if(len(y_true.shape) == 1):
        return np.array([np.log(y_pred[i]) for i in y_true])
    #if onehot encoded
    elif(len(y_true.shape) == 2):
        return np.array([-(y_true[i] * np.log(y_pred[i])).sum() for i in range(len(y_pred))])
    
def softmax(x):
    x = np.array(x)
    return np.exp(x) / np.exp(x).sum(axis=1).reshape(x.shape[0],-1)

"""
get a first Idea of how the Models perform on the inital data (at this time I didn't know that we would get results in the email as well)
"""
#Encode sexes using one-hot encoding and create training data and labels
x = np.concatenate((getOneHotEncoding(full_sexes), initial_molluscs_data.values[:,1:-1]), axis=1)
y = getOneHotEncoding(full_stages)

print("mlp")
mlp = MLPClassifier(max_iter=1000)
mlp.fit(x[:10], y[:10])
y_pred = softmax(mlp.predict_proba(x[11:]))
print(f"predicted: {y_pred} real: {y[11:]}")
mlp = MLPClassifier(max_iter=1000)
#see how good the model performs using cross entropy as the metric (code doesn't work anymore and i'm too lazy to see why and fix it, I hope the idea that is presented is enough)
#cvs_mlp = cross_val_score(mlp, x, y, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_mlp)

#encode labels as numbers for SVC and RFC
y_rfc = full_stages.replace("Adult",0).replace("Adole",1).replace("Child",2)

print("svc")
svc = SVC(kernel="rbf", probability=True)
svc.fit(x[:10], y_rfc[:10])
y_pred = softmax(svc.predict_proba(x[11:]))
print(f"predicted: {y_pred} real: {y[11:]}")
svc = SVC(kernel="rbf", probability=True)
#see how good the model performs using cross entropy as the metric
#cvs_svc = cross_val_score(svc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_svc)

print("rfc")
rfc = RandomForestClassifier()
rfc.fit(x[:10], y_rfc[:10])
y_pred = softmax(rfc.predict_proba(x[11:]))
print(f"predicted: {y_pred} real: {y[11:]}")
rfc = RandomForestClassifier()
#see how good the model performs using cross entropy as the metric
#cvs_rfc = cross_val_score(rfc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(f"{cvs_rfc=}")

#First idea for an objective function for baysian optimization
def objectiveFunction(x, y):
    rfc = RandomForestClassifier()
    cvs_rfc = cross_val_score(rfc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
    svc = SVC(kernel="rbf", probability=True)
    cvs_svc = cross_val_score(svc, x, y_rfc, cv=3, scoring=score_cross_entropy_loss)
    mlp = MLPClassifier(max_iter=1000)
    cvs_mlp = cross_val_score(mlp, x, y, cv=3, scoring=score_cross_entropy_loss)
    return (cvs_rfc.mean() + cvs_svc.mean() + cvs_mlp.mean()) / 3




mlp
predicted: [[0.42943023 0.37444436 0.19612541]
 [0.20951428 0.56118949 0.22929623]
 [0.29903595 0.43849029 0.26247376]
 [0.24140385 0.44460473 0.31399142]
 [0.21586358 0.31799939 0.46613703]] real: [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
svc
predicted: [[0.40766389 0.27408039 0.31825572]
 [0.39480692 0.32064965 0.28454343]
 [0.39655424 0.29753548 0.30591028]
 [0.40175276 0.32239626 0.27585098]
 [0.40758887 0.3233908  0.26902032]] real: [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
rfc
predicted: [[0.39134195 0.37599722 0.23266083]
 [0.43409497 0.23119542 0.33470961]
 [0.4298035  0.3121014  0.2580951 ]
 [0.45558268 0.24507817 0.29933915]
 [0.38365173 0.23269654 0.38365173]] real: [[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


c:\Users\ben-g\anaconda3\envs\e2ml-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
"""
Get the first batch of data.
Since time was close and I needed to get data, I opted to just draw samples randomly.
Sexes were choses from the original distribution by drawing with placing back.
Here I made the assumption that the volume and the weight of the mollusc are dependent on each other, and I could calculate the weight by multiplying the weigth by a quotient with some noise.
To achieve this I fitted a normal distribution with the qutient of the molluscs I got from the inital data.
I also made the assumption that the volume and thus the length, width and height would be normaly distributed as often time in nature this is the case (height for humans for example).
I used the inital data to parameterize these normal distributions and then drew samples from them to get the data for the first batch.
I also checked the mean and standard deviation of the new data and some extra statistics to see if it would actually match those of the old data.
"""

def getNewSamples(old_data:pd.Series, size:int):
    return np.random.normal(old_data.mean(), scale=old_data.std(), size=size)

def getDict(size:int):
    """
    Just get a dict with the values that can be turned into a dataframe using pandas
    """
    d = {}
    d["Sex"] = np.random.choice(full_sexes, size)
    d["Length"] = getNewSamples(full_length, size)
    d["Width"] = getNewSamples(full_width, size)
    d["Height"] =  getNewSamples(full_height, size)
    d["Weight"] = d["Height"] * d["Width"] * d["Length"] * np.random.normal(weight_volume_quotients.mean(), weight_volume_quotients.std())
    d["Non_Shell Weight"] = d["Weight"] * np.random.normal(non_shell_quotient.mean(), non_shell_quotient.std())
    d["Intestine Weight"] = d["Weight"] * np.random.normal(intestine_quotient.mean(), intestine_quotient.std())
    d["Shell Weight"] = d["Weight"] * np.random.normal(shell_quotient.mean(), shell_quotient.std())
    return d
d = getDict(282)
print(f"{d['Weight'].mean()=} {d['Weight'].std()=}")
print(f"{full_weight.mean()=} {full_weight.std()=}")
print(f"{d['Non_Shell Weight'].mean()=} {d['Non_Shell Weight'].std()=}")
print(f"{full_non_shell_weight.mean()=} {full_non_shell_weight.std()=}")
print(f"{d['Intestine Weight'].mean()=} {d['Intestine Weight'].std()=}")
print(f"{full_intestine_weight.mean()=} {full_intestine_weight.std()=}")
print(f"{d['Shell Weight'].mean()=} {d['Shell Weight'].std()=}")
print(f"{full_shell_weight.mean()=} {full_shell_weight.std()=}")
orignial_diffs = full_weight - full_intestine_weight - full_non_shell_weight - full_shell_weight
new_diffs = d["Weight"] - d["Intestine Weight"] - d["Non_Shell Weight"] - d["Shell Weight"]
print(orignial_diffs)
print(new_diffs)
print(f"{orignial_diffs.mean()=} {orignial_diffs.std()=}")
print(f"{new_diffs.mean()=} {new_diffs.std()=}")
print(pd.DataFrame(d))

In [6]:
print(pd.read_csv("FirstBatchBen.csv"))

     Unnamed: 0 Sex    Length     Width    Height    Weight  Non_Shell Weight  \
0             0   M  0.471305  0.290373  0.106767  0.303367          0.123269   
1             1   F  0.731545  0.478463  0.170384  1.238197          0.503123   
2             2   I  0.615436  0.235762  0.065890  0.198494          0.080655   
3             3   F  0.451397  0.144832  0.161660  0.219429          0.089162   
4             4   F  0.689066  0.398501  0.175774  1.002112          0.407194   
..          ...  ..       ...       ...       ...       ...               ...   
277         277   F  0.555961  0.375405  0.131120  0.568176          0.230870   
278         278   F  0.605268  0.583672  0.065907  0.483415          0.196429   
279         279   M  0.529225  0.241043  0.140752  0.372786          0.151476   
280         280   F  0.411005  0.499416  0.121998  0.519914          0.211260   
281         281   M  0.603832  0.480075  0.120224  0.723582          0.294017   

     Intestine Weight  Shel

In [7]:
"""
My idea for getting new data from the beginning was to use baysian optimization (BO).
The Idea is using cross_entropy as a metric on how well the model performs and use that to perform BO.
The first big problem I encountered was the problem of dimensionality.
It was not possible to use the original feature space with all 7 dimensions to create a good amount of candidates.
For example if I took only 100 possible values for each feature I would get 100^8 = 1e16 possible combinations (about 10 Petaabyte if float32 used and my math is not off).
That wouldn't fit into my ram. I could reduce the number possible values but that would limit the range of samples I could get drasticly and make it hard for the BO to actually find good samples for the next batch of data
To solve that problem I took a look into how good PCA works for dimensionality reduction.
The first 2 prinicpled components were responsible for 88% of the variance in the dataset, so pca looked like a good choice to reduce the dimensionality to a good amount for BO and still keep most of the information present in the data.
It also made sense to me, that pca would perform well, as I already had the idea that since most of the features were dependent on each other (full weight ~ Non_Shell Weight + Intestine Weight + Shell Weight, width*length*height ~ weight) a lot of varaince should be explainable by a lower number of features of some form.
Since pca doesn't work on categorical data (sex) the sexes are represented as 0, 1, 2 and included in this way here. Good idea? We will see...
"""

full_replaced = initial_molluscs_data.replace("F",0).replace("I",1).replace("M",2)
x_full_replaced = full_replaced.values[:,:-1]
pca = PCA(2)
pca = pca.fit(x_full_replaced)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

#look at the range of the values of the pca to determine the range of the candidates. In this case the range is (-1.5, 1.5)
x_pca = pca.transform(x_full_replaced)
print(x_pca)

#new valuerange is -1.5, 1.5

#look at performance of models if using the pca values
print("mlp")
mlp = MLPClassifier(max_iter=1000)
mlp.fit(x_pca[:10], y[:10])
y_pred = mlp.predict_proba(x_pca[11:])
y_pred = softmax(y_pred)
mlp = MLPClassifier(max_iter=1000)
#cvs_mlp = cross_val_score(mlp, x_pca, y, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_mlp)

y_rfc = full_stages.replace("Adult",0).replace("Adole",1).replace("Child",2)

print("svc")
svc = SVC(kernel="rbf", probability=True)
svc.fit(x_pca[:10], y_rfc[:10])
y_pred = softmax(svc.predict_proba(x_pca[11:]))
svc = SVC(kernel="rbf", probability=True)
#cvs_svc = cross_val_score(svc, x_pca, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(cvs_svc)

print("rfc")

rfc = RandomForestClassifier()
rfc.fit(x_pca[:10], y_rfc[:10])
y_pred = softmax(rfc.predict_proba(x_pca[11:]))
rfc = RandomForestClassifier()
#cvs_rfc = cross_val_score(rfc, x_pca, y_rfc, cv=3, scoring=score_cross_entropy_loss)
#print(f"{cvs_rfc=}")

def objectiveFunctionRFC(x, y_rfc):
    rfc = RandomForestClassifier()
    rfc.fit(x,y_rfc)
    return score_cross_entropy_loss(rfc, x, y_rfc)

def objectiveFunctionSVC(x,y_rfc):
    svc = SVC(kernel="rbf", probability=True)
    svc.fit(x, y_rfc)
    return score_cross_entropy_loss(svc, x, y_rfc)

def objectiveFunctionMLP(x, y):
    mlp = MLPClassifier(max_iter=1000)
    mlp.fit(x,y)
    return score_cross_entropy_loss(mlp, x, y)


x_acquired = x_pca
y_acquired_rfc = objectiveFunctionRFC(x_acquired, y_rfc)
print(y_acquired_rfc)
y_acquired_svc = objectiveFunctionSVC(x_pca, y_rfc)
print(y_acquired_svc)
y_acquired_mlp = objectiveFunctionMLP(x_pca, y)
print(y_acquired_mlp)

metrics_dict = {'gamma': 50, 'metric': 'rbf'}
gpr = GaussianProcessRegression(metrics_dict=metrics_dict)
gpr.fit(x_acquired, y_acquired_rfc)

x1_new = np.linspace(-1.5, 1.5, 100)
x_mesh,y_mesh = np.meshgrid(x1_new, x1_new)

x_cand = np.stack((x_mesh, y_mesh), axis=2).reshape(-1,2)
print(x_cand.shape)
means, stds = gpr.predict(x_cand, True)
scores = acquisition_ei(means, stds, max(y_acquired_rfc))
print(scores)
print(scores.mean())
print(scores.std())
print(scores.min())
print(scores.max())

nextidx = np.argsort(scores)

print(scores[nextidx[-10:]])
print(x_cand[nextidx[-10:]])
print(pca.inverse_transform(x_cand[nextidx[-10:]]))

[0.70537116 0.28817022]
[3.54073097 2.26312547]
[[-0.74840581 -0.62740424]
 [-0.76548386 -0.47050949]
 [ 1.15757455  0.32628467]
 [ 0.24195993 -0.4856955 ]
 [ 1.29797218 -0.90029362]
 [ 1.12356131  0.64986098]
 [ 0.22758857 -0.36834205]
 [-0.84014791  0.2007329 ]
 [-0.90098976  0.77220098]
 [-0.83319121  0.13402528]
 [-0.79940449 -0.16651914]
 [ 0.2713473  -0.7486818 ]
 [-0.92209266  0.9578332 ]
 [-0.78875156 -0.27996054]
 [ 1.15739731  0.34398507]
 [ 1.12106611  0.66248328]]
mlp


c:\Users\ben-g\anaconda3\envs\e2ml-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


svc
rfc
[-0.4773645723611691, -0.4098049359074743, -0.3729031519548653, -0.530239431883584, -0.35372802568051775, -0.39291368919781977, -0.4762058457221013, -0.5027245280704169, -0.47696637052782237, -0.4979112796885527, -0.37696525203047004, -0.40095331281649926, -0.4731118341069962, -0.3690954346579947, -0.3729031519548653, -0.39291368919781977]
[-0.515057829735693, -0.7002840914250089, -0.6009751991975496, -0.5760999613352049, -0.6257006415990883, -0.658015775472558, -0.6171860047097263, -0.7230883435484797, -0.5740028297436728, -0.5296547225791984, -0.6975862318597081, -0.6279255232347337, -0.6186053714378043, -0.7013980834181033, -0.5998145419708573, -0.6585480373443983]


c:\Users\ben-g\anaconda3\envs\e2ml-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[-0.5164647053346821, -0.47450517476205495, -0.3590114280451305, -0.6232765364671853, -0.34353638415286, -0.36449831160723517, -0.530208813762346, -0.5979227025085164, -0.501828882620654, -0.5810820576470737, -0.43785968140227904, -0.3850810852009933, -0.4856460443119528, -0.38861726717627904, -0.36477844752264693, -0.3606014681718168]
(10000, 2)
[0.60069514 0.60069514 0.60069514 ... 0.60069514 0.60069514 0.60069514]
0.5703279326726639
0.08902937074562546
6.890552061449001e-06
0.6006951355493214
[0.60069514 0.60069514 0.60069514 0.60069514 0.60069514 0.60069514
 0.60069514 0.60069514 0.60069514 0.60069514]
[[0.13636364 1.04545455]
 [0.28787879 1.01515152]
 [0.22727273 1.01515152]
 [0.1969697  1.01515152]
 [0.16666667 1.01515152]
 [0.13636364 1.01515152]
 [0.10606061 1.01515152]
 [0.07575758 1.01515152]
 [0.25757576 1.01515152]
 [1.5        1.5       ]]
[[1.06268547 0.72025198 0.58817507 0.20941003 1.74205705 0.7599597
  0.37273336 0.48780996]
 [1.20990022 0.70836333 0.57865376 0.205718